Stacked Autoencoder : A stacked autoencoder is a neural network that consists of several layers of sparse autoencoders where output of each hidden layer is connected to the input of the successive hidden layer.


In [135]:
# Importing the libraries :

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [136]:
# Importing the dataset :

movies = pd.read_csv('/content/drive/MyDrive/AutoEncoders/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('/content/drive/MyDrive/AutoEncoders/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('/content/drive/MyDrive/AutoEncoders/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')


In [137]:
print(movies[:5])

   0                                   1                             2
0  1                    Toy Story (1995)   Animation|Children's|Comedy
1  2                      Jumanji (1995)  Adventure|Children's|Fantasy
2  3             Grumpier Old Men (1995)                Comedy|Romance
3  4            Waiting to Exhale (1995)                  Comedy|Drama
4  5  Father of the Bride Part II (1995)                        Comedy


In [138]:
print(users[:5])

   0  1   2   3      4
0  1  F   1  10  48067
1  2  M  56  16  70072
2  3  M  25  15  55117
3  4  M  45   7  02460
4  5  M  25  20  55455


In [139]:
print(ratings[:5])

   0     1  2          3
0  1  1193  5  978300760
1  1   661  3  978302109
2  1   914  3  978301968
3  1  3408  4  978300275
4  1  2355  5  978824291


In [140]:
#Training and Test Sets :

training_set = pd.read_csv('/content/drive/MyDrive/AutoEncoders/ml-1m/training_set.csv', delimiter='\t')
training_set = np.array([row.split(',') for row in training_set['User,Movie,Rating,Timestamp']], dtype=int)


test_set = pd.read_csv('/content/drive/MyDrive/AutoEncoders/ml-1m/test_set.csv', delimiter='\t')
test_set = np.array([row.split(',') for row in test_set['User,Movie,Rating,Timestamp']], dtype=int)


In [141]:
print(training_set[:5])

[[        1       661         3 978302109]
 [        1       914         3 978301968]
 [        1      3408         4 978300275]
 [        1      2355         5 978824291]
 [        1      1287         5 978302039]]


In [142]:
print(test_set[:5])

[[        1      1193         5 978300760]
 [        1      1197         3 978302268]
 [        1      2804         5 978300719]
 [        1       595         5 978824268]
 [        1       938         4 978301752]]


In [143]:
# Getting the number of users and movies

nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

print(nb_users)
print(nb_movies)

6040
3952


In [144]:
# Converting the data into an array with users in lines and movies in columns

def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data


training_set = convert(training_set)
test_set = convert(test_set)

In [155]:
print(training_set[0])

tensor([5., 0., 0.,  ..., 0., 0., 0.])


In [149]:
# Converting the data into Torch tensors

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [151]:
# Creating the architecture of the Neural Network"""

class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [152]:
# Training the SAE

nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.3478)
epoch: 2loss: tensor(1.0100)
epoch: 3loss: tensor(0.9902)
epoch: 4loss: tensor(0.9833)
epoch: 5loss: tensor(0.9801)
epoch: 6loss: tensor(0.9786)
epoch: 7loss: tensor(0.9772)
epoch: 8loss: tensor(0.9766)
epoch: 9loss: tensor(0.9760)
epoch: 10loss: tensor(0.9757)
epoch: 11loss: tensor(0.9753)
epoch: 12loss: tensor(0.9750)
epoch: 13loss: tensor(0.9749)
epoch: 14loss: tensor(0.9747)
epoch: 15loss: tensor(0.9743)
epoch: 16loss: tensor(0.9743)
epoch: 17loss: tensor(0.9743)
epoch: 18loss: tensor(0.9741)
epoch: 19loss: tensor(0.9739)
epoch: 20loss: tensor(0.9739)
epoch: 21loss: tensor(0.9738)
epoch: 22loss: tensor(0.9736)
epoch: 23loss: tensor(0.9735)
epoch: 24loss: tensor(0.9729)
epoch: 25loss: tensor(0.9724)
epoch: 26loss: tensor(0.9717)
epoch: 27loss: tensor(0.9708)
epoch: 28loss: tensor(0.9702)
epoch: 29loss: tensor(0.9690)
epoch: 30loss: tensor(0.9678)
epoch: 31loss: tensor(0.9677)
epoch: 32loss: tensor(0.9675)
epoch: 33loss: tensor(0.9662)
epoch: 34loss: tens

In [154]:
# Testing the SAE

test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9217)


We got a test loss of 0.92 i.e. our model is
going to predict a rating that will be different from
the real rating by less than one star.
Our recommended system can predict for new movies
if we are going to like it or not ie it will predict a rating close to the real rating.
